In [2]:
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt

In [3]:
conn_string = "host='localhost' dbname='stockmarket' user='postgres' password='5265104d'"
conn = psycopg2.connect(conn_string)

In [4]:
df = pd.read_sql_query(
    '''select i.date ,i.open ,i.high ,i.low ,i.close ,i.adj_close ,i.volume ,c.name
    from dataset_information i 
    join dataset_company c
    on i.company_id=c.id
    where company_id = {}'''.format(3)  , conn, index_col='date')

##### To calculate the MAAC  we have to calculate 
1. Calculate a 12 day EMA of closing prices
2. Calculate a 26 day EMA of closing prices
3. Subtract the longer EMA in (2) from the shorter EMA in (1)
4. Calculate a 9 day EMA of the MACD line gotten in (3)

##### calculate 26 Exponintial moving average

In [11]:
df['26 ema'] = pd.ewma(df["adj_close"], span=26)
df

C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(span=26,min_periods=0,adjust=True,ignore_na=False).mean()
  """Entry point for launching an IPython kernel.


,open,high,low,close,adj_close,volume,name,26 ema
date,,,,,,,,
1986-03-12,0.444196,0.448661,0.441964,0.441964,0.020108,21420000.0,AAPL,0.020108
1986-03-13,0.441964,0.446429,0.435268,0.441964,0.020108,28991200.0,AAPL,0.020108
1986-03-14,0.441964,0.468750,0.441964,0.466518,0.021225,96213600.0,AAPL,0.020509
1986-03-17,0.464286,0.464286,0.453125,0.464286,0.021123,29680000.0,AAPL,0.020681
1986-03-18,0.464286,0.486607,0.462054,0.479911,0.021834,62339200.0,AAPL,0.020948
1986-03-19,0.479911,0.486607,0.470982,0.473214,0.021529,47471200.0,AAPL,0.021065
1986-03-20,0.500000,0.529018,0.500000,0.504464,0.022951,226032800.0,AAPL,0.021400
1986-03-21,0.504464,0.513393,0.491071,0.493304,0.022443,65094400.0,AAPL,0.021568
1986-03-24,0.493304,0.493304,0.470982,0.477679,0.021733,73578400.0,AAPL,0.021593


##### calculate 12 Exponintial moving average

In [12]:
df['12 ema'] = pd.ewma(df["adj_close"], span=12)
df

C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(span=12,min_periods=0,adjust=True,ignore_na=False).mean()
  """Entry point for launching an IPython kernel.


,open,high,low,close,adj_close,volume,name,26 ema,12 ema
date,,,,,,,,,
1986-03-12,0.444196,0.448661,0.441964,0.441964,0.020108,21420000.0,AAPL,0.020108,0.020108
1986-03-13,0.441964,0.446429,0.435268,0.441964,0.020108,28991200.0,AAPL,0.020108,0.020108
1986-03-14,0.441964,0.468750,0.441964,0.466518,0.021225,96213600.0,AAPL,0.020509,0.020544
1986-03-17,0.464286,0.464286,0.453125,0.464286,0.021123,29680000.0,AAPL,0.020681,0.020727
1986-03-18,0.464286,0.486607,0.462054,0.479911,0.021834,62339200.0,AAPL,0.020948,0.021028
1986-03-19,0.479911,0.486607,0.470982,0.473214,0.021529,47471200.0,AAPL,0.021065,0.021149
1986-03-20,0.500000,0.529018,0.500000,0.504464,0.022951,226032800.0,AAPL,0.021400,0.021551
1986-03-21,0.504464,0.513393,0.491071,0.493304,0.022443,65094400.0,AAPL,0.021568,0.021738
1986-03-24,0.493304,0.493304,0.470982,0.477679,0.021733,73578400.0,AAPL,0.021593,0.021737


#### calculate MAACD

In [15]:
df['MACD'] = (df['12 ema'] - df['26 ema'])
print(df.head())

                open      high       low     close  adj_close      volume  \
date                                                                        
1986-03-12  0.444196  0.448661  0.441964  0.441964   0.020108  21420000.0   
1986-03-13  0.441964  0.446429  0.435268  0.441964   0.020108  28991200.0   
1986-03-14  0.441964  0.468750  0.441964  0.466518   0.021225  96213600.0   
1986-03-17  0.464286  0.464286  0.453125  0.464286   0.021123  29680000.0   
1986-03-18  0.464286  0.486607  0.462054  0.479911   0.021834  62339200.0   

            name    26 ema    12 ema      MACD  
date                                            
1986-03-12  AAPL  0.020108  0.020108  0.000000  
1986-03-13  AAPL  0.020108  0.020108  0.000000  
1986-03-14  AAPL  0.020509  0.020544  0.000035  
1986-03-17  AAPL  0.020681  0.020727  0.000046  
1986-03-18  AAPL  0.020948  0.021028  0.000079  


### plotting  close against MACD